# 8-puzzle: IA 1 - Professor Li Weigang
### Pedro Araujo - 190050560 - 08/04/2019
#### Adaptado de:
### Fred Guth - 190050411 - 21/03/2019

In [13]:
import numpy as np
from sortedcontainers import SortedDict
from collections import OrderedDict
import heapq

__moves__ guarda as movimentações (swaps) possíveis para cada posição do tabuleiro.
O tabuleiro é representado por uma simples lista ou um número. 
Por exemplo, a posição inicial é o estado [5,4,0,6,1,8,7,3,2] e também pode ser representado por 540618732.

In [14]:
moves=np.array([[1,3],[0,2,4],[1,5],[1,4,6],[1,3,5,7],[2,4,8],[3,7],[6,4,8],[5,7]])
initial = [2,8,3,1,6,4,7,0,5] # 0 é o espaço vazio
final = [1,2,3,8,0,4,7,6,5]
visited = SortedDict()

In [15]:
# "Troca" alguma posição do tabuleiro com o espaço vazio.  Não verifica se o swap é válido.
def Swap(state, pos):
    empty = state.index(0)
    state[empty], state[pos] = state[pos], state[empty]
    return state

In [16]:
# Imprime o tabuleiro
def PrintBoard(state):
    print (np.asarray(state).reshape((3,3)))

In [17]:
# Transforma a representação do tabuleiro por lista em representação por número
#  [0, 2, 3, 8, 4, 1, 7, 6, 5] => 23841765
def Numerify(state):
    return int("".join(map(str, state[:]))) 

In [18]:
# Transforma a representação do tabuleiro por lista em representação por número
# 23841765 => [0, 2, 3, 8, 4, 1, 7, 6, 5]
def Listify(number):
    list =  [int(x) for x in str(number)]
    if (len(list)<9):
        list = [0] + list
    return list

In [19]:
# Imprime todas as posições do tabuleiro do estado inicial ao "state"
def Trace(state, visited):
    stack = []
    stack.append(state)
    parent = visited[Numerify(state)]['parent']
    while (parent is not None):
        stack.insert(0, parent)
        parent = visited[Numerify(parent)]['parent']
    for i, s in enumerate(stack):
        print(f'position #{i}:')
        PrintBoard(s)

In [143]:
def TraceTree(depth, visited):
    tree = {}
    for i in visited:
        if (visited[i]["moves"] in tree):
            tree[visited[i]["moves"]].append(i)
        else:
            tree[visited[i]["moves"]]=[i]
    for j in range(0, len(tree)):
        print("-----------------")
        print("depth:", j)
        for k in tree[j]:
            PrintBoard(Listify(k))
            if (k == Numerify(final)):
                print ('*')


In [139]:
def isGoal(state, goal):
    return Numerify(state)  == Numerify(goal)

## Primeira Abordagem: BFS
O problema do Breadth-First Search é que não há nenhuma priorização entre os estados enfileirados para serem buscados.

In [146]:
# Breadth-First Search
def BFS(source):
    counter = 0
    visited = {}
    visited[Numerify(source)]={'moves': 0, 'parent': None}
    if isGoal(source, final):
        print (f'{counter:,} evaluations.')
        print (visited[Numerify(source)])
        return
    sourceHash = Numerify(source)
    frontier = OrderedDict()
    frontier[sourceHash] = 0
    while True:
        if len(frontier) == 0:
            return "FAILURE"
        node = Listify(frontier.popitem(last=False)[0])
        g = visited[Numerify(node[:])]['moves'] # m = g(state) = moves to current state
        empty = node.index(0) # where is empty
        for a in moves[empty]:
            child = Swap(node[:], a) # u[:] makes a copy of u
            if Numerify(child) not in visited and Numerify(child) not in frontier:
                visited[Numerify(child)]={'moves':g+1, 'parent': node}
                if isGoal(child, final):
                        print (f'{counter:,} evaluations.')
                        print (visited[Numerify(child)])
                        return visited
                else:
                    counter = counter + 1
                    frontier[Numerify(child)] = 0
    return visited

In [147]:
visited = BFS(initial)

48 evaluations.
{'moves': 5, 'parent': [1, 2, 3, 0, 8, 4, 7, 6, 5]}


In [148]:
TraceTree(5,visited)

-----------------
depth: 0
[[2 8 3]
 [1 6 4]
 [7 0 5]]
-----------------
depth: 1
[[2 8 3]
 [1 6 4]
 [0 7 5]]
[[2 8 3]
 [1 0 4]
 [7 6 5]]
[[2 8 3]
 [1 6 4]
 [7 5 0]]
-----------------
depth: 2
[[2 8 3]
 [0 6 4]
 [1 7 5]]
[[2 0 3]
 [1 8 4]
 [7 6 5]]
[[2 8 3]
 [0 1 4]
 [7 6 5]]
[[2 8 3]
 [1 4 0]
 [7 6 5]]
[[2 8 3]
 [1 6 0]
 [7 5 4]]
-----------------
depth: 3
[[2 0 3]
 [8 6 4]
 [1 7 5]]
[[2 8 3]
 [6 0 4]
 [1 7 5]]
[[0 2 3]
 [1 8 4]
 [7 6 5]]
[[2 3 0]
 [1 8 4]
 [7 6 5]]
[[2 0 3]
 [8 1 4]
 [7 6 5]]
[[2 8 3]
 [7 1 4]
 [0 6 5]]
[[2 8 0]
 [1 4 3]
 [7 6 5]]
[[2 8 3]
 [1 4 5]
 [7 6 0]]
[[2 8 0]
 [1 6 3]
 [7 5 4]]
[[2 8 3]
 [1 0 6]
 [7 5 4]]
-----------------
depth: 4
[[0 2 3]
 [8 6 4]
 [1 7 5]]
[[2 3 0]
 [8 6 4]
 [1 7 5]]
[[2 6 3]
 [8 0 4]
 [1 7 5]]
[[2 0 3]
 [6 8 4]
 [1 7 5]]
[[2 8 3]
 [6 4 0]
 [1 7 5]]
[[2 8 3]
 [6 7 4]
 [1 0 5]]
[[1 2 3]
 [0 8 4]
 [7 6 5]]
[[2 3 4]
 [1 8 0]
 [7 6 5]]
[[0 2 3]
 [8 1 4]
 [7 6 5]]
[[2 3 0]
 [8 1 4]
 [7 6 5]]
[[2 1 3]
 [8 0 4]
 [7 6 5]]
[[2 8 3]
 [7 1 4]
 [6 0 5

In [149]:
Trace(final, visited)

position #0:
[[2 8 3]
 [1 6 4]
 [7 0 5]]
position #1:
[[2 8 3]
 [1 0 4]
 [7 6 5]]
position #2:
[[2 0 3]
 [1 8 4]
 [7 6 5]]
position #3:
[[0 2 3]
 [1 8 4]
 [7 6 5]]
position #4:
[[1 2 3]
 [0 8 4]
 [7 6 5]]
position #5:
[[1 2 3]
 [8 0 4]
 [7 6 5]]


## Segunda Abordagem: DFS

In [150]:
# Depth-First Search
def DFS(source):
    counter = 0
    visited = {}
    visited[Numerify(source)]={'moves': 0, 'parent': None}
    if isGoal(source, final):
        print (f'{counter:,} evaluations.')
        print (visited[Numerify(source)])
        return
    sourceHash = Numerify(source)
    frontier = OrderedDict()
    frontier[sourceHash] = 0
    while True:
        if len(frontier) == 0:
            return "FAILURE"
        node = Listify(frontier.popitem(last=True)[0])
        g = visited[Numerify(node[:])]['moves'] # m = g(state) = moves to current state
        empty = node.index(0) # where is empty
        for a in moves[empty]:
            child = Swap(node[:], a) # u[:] makes a copy of u
            if Numerify(child) not in visited and Numerify(child) not in frontier:
                visited[Numerify(child)]={'moves':g+1, 'parent': node}
                if isGoal(child, final):
                        print (f'{counter:,} evaluations.')
                        print (visited[Numerify(child)])
                        return visited
                else:
                    counter = counter + 1
                    frontier[Numerify(child)] = 0
    return visited

In [151]:
visited = DFS(initial)

226,436 evaluations.
{'moves': 112667, 'parent': [1, 2, 3, 0, 8, 4, 7, 6, 5]}


## Terceira Abordagem: A*

In [153]:
import heapq

In [152]:
def manhattan_distance(current, goal):
    return sum(abs(b%3 - g%3) + abs(b//3 - g//3)
        for b, g in ((current.index(i), goal.index(i)) for i in range(1, 9)))

In [156]:
# A_star: usaremos um heap para fila de prioridade e a função *heuristic* para determinar o custo
def A_star(heuristic, source):
#         for v in moves[empty]:
#             state = Swap(u[:], v) # u[:] makes a copy of u
#             counter = counter + 1
#             if not(visited.__contains__(Numerify(state))):
#                 visited[Numerify(state)]={'moves':g+1, 'parent': u}
#                 if (Numerify(state) == Numerify(final)):
#                         print (f'{counter:,} evaluations.')
#                         print (visited[Numerify(state)])
#                         return
#                 else:
#                     h = heuristic(state, final)
#                     heapq.heappush(queue, ((g + h), state)) # f(state)= g(state)+h(state)
    counter = 0
    visited = {}
    visited[Numerify(source)]={'moves': 0, 'parent': None}
    frontier = []
    heapq.heappush(frontier, (0, source))
    while True:
        if len(frontier) == 0:
            return "FAILURE"
        node = heapq.heappop(frontier)[1]
        if isGoal(node, final):
            print (f'{counter:,} evaluations.')
            print (visited[Numerify(child)])
            return visited
        g = visited[Numerify(node[:])]['moves'] # m = g(state) = moves to current state
        empty = node.index(0) # where is empty
        for a in moves[empty]:
            child = Swap(node[:], a) # u[:] makes a copy of u
            if Numerify(child) not in visited and Numerify(child) not in frontier:
                visited[Numerify(child)]={'moves':g+1, 'parent': node}
                counter = counter + 1
                h = heuristic(child, final)
                heapq.heappush(frontier, ((g + h), child))
    return visited

In [157]:
visited = A_star(manhattan_distance, initial)

12 evaluations.
{'moves': 5, 'parent': [1, 2, 3, 0, 8, 4, 7, 6, 5]}


In [159]:
TraceTree(final, visited)

-----------------
depth: 0
[[2 8 3]
 [1 6 4]
 [7 0 5]]
-----------------
depth: 1
[[2 8 3]
 [1 6 4]
 [0 7 5]]
[[2 8 3]
 [1 0 4]
 [7 6 5]]
[[2 8 3]
 [1 6 4]
 [7 5 0]]
-----------------
depth: 2
[[2 0 3]
 [1 8 4]
 [7 6 5]]
[[2 8 3]
 [0 1 4]
 [7 6 5]]
[[2 8 3]
 [1 4 0]
 [7 6 5]]
-----------------
depth: 3
[[0 2 3]
 [1 8 4]
 [7 6 5]]
[[2 3 0]
 [1 8 4]
 [7 6 5]]
-----------------
depth: 4
[[1 2 3]
 [0 8 4]
 [7 6 5]]
-----------------
depth: 5
[[1 0 3]
 [2 8 4]
 [7 6 5]]
[[1 2 3]
 [8 0 4]
 [7 6 5]]
*
[[1 2 3]
 [7 8 4]
 [0 6 5]]


In [158]:
Trace(final, visited)

position #0:
[[2 8 3]
 [1 6 4]
 [7 0 5]]
position #1:
[[2 8 3]
 [1 0 4]
 [7 6 5]]
position #2:
[[2 0 3]
 [1 8 4]
 [7 6 5]]
position #3:
[[0 2 3]
 [1 8 4]
 [7 6 5]]
position #4:
[[1 2 3]
 [0 8 4]
 [7 6 5]]
position #5:
[[1 2 3]
 [8 0 4]
 [7 6 5]]
